In [ ]:
%serialconnect to port=/dev/tty.usbserial-0001 --baud=115200

In [26]:
from machine import Pin, SoftI2C
from time import sleep_ms

import os

mpu_addr = 0x68
mpu_pwr_mgmt = 0x6B

mpu_accel_x_h = 0x3B
mpu_accel_x_l = 0x3C
mpu_accel_y_h = 0x3D
mpu_accel_y_l = 0x3E
mpu_accel_z_h = 0x3F
mpu_accel_z_l = 0x40

mpu_gyro_x_h = 0x43
mpu_gyro_x_l = 0x44
mpu_gyro_y_h = 0x45
mpu_gyro_y_l = 0x46
mpu_gyro_z_h = 0x47
mpu_gyro_z_l = 0x48

mpu_lsbc = 340.0
mpu_lsbg = 16348.0

g = 9.8066

def mpu_init(i2c):
    i2c.writeto_mem(mpu_addr, mpu_pwr_mgmt, bytes([0]))

def combine_register_values(h, l):
    if not h[0] & 0x80:
        return h[0] << 8 | l[0]
    return -((h[0] ^ 255) << 8) |  (l[0] ^ 255) + 1

def mpu_get_accel(i2c):
    accel_x_h = i2c.readfrom_mem(mpu_addr, mpu_accel_x_h, 1)
    accel_x_l = i2c.readfrom_mem(mpu_addr, mpu_accel_x_l, 1)
    accel_y_h = i2c.readfrom_mem(mpu_addr, mpu_accel_y_h, 1)
    accel_y_l = i2c.readfrom_mem(mpu_addr, mpu_accel_y_l, 1)
    accel_z_h = i2c.readfrom_mem(mpu_addr, mpu_accel_z_h, 1)
    accel_z_l = i2c.readfrom_mem(mpu_addr, mpu_accel_z_l, 1)

    return [accel_correction(combine_register_values(accel_x_h, accel_x_l)),
            accel_correction(combine_register_values(accel_y_h, accel_y_l)),
            accel_correction(combine_register_values(accel_z_h, accel_z_l))]

def accel_correction(accel):
    return accel / mpu_lsbg

def mpu_get_gyro(i2c):
    gyro_x_h = i2c.readfrom_mem(mpu_addr, mpu_gyro_x_h, 1)
    gyro_x_l = i2c.readfrom_mem(mpu_addr, mpu_gyro_x_l, 1)
    gyro_y_h = i2c.readfrom_mem(mpu_addr, mpu_gyro_y_h, 1)
    gyro_y_l = i2c.readfrom_mem(mpu_addr, mpu_gyro_y_l, 1)
    gyro_z_h = i2c.readfrom_mem(mpu_addr, mpu_gyro_z_h, 1)
    gyro_z_l = i2c.readfrom_mem(mpu_addr, mpu_gyro_z_l, 1)

    return [combine_register_values(gyro_x_h, gyro_x_l) / mpu_lsbc,
            combine_register_values(gyro_y_h, gyro_y_l) / mpu_lsbc,
            combine_register_values(gyro_z_h, gyro_z_l) / mpu_lsbc]

if __name__ == "__main__":
    i2c = SoftI2C(scl=Pin(23), sda=Pin(22))
    mpu_init(i2c)
    
    while True:
        print("Accelerometer:\t", mpu_get_accel(i2c), "g")
        print("Gyroscope:\t", mpu_get_gyro(i2c), "°/s")
        sleep_ms(1000)


Accelerometer:	 [0.1707854, 0.2495718, 1.154147] g
Gyroscope:	 [7.4, -2.155882, 4.291176] °/s
Accelerometer:	 [-0.7614388, -0.401517, -0.8514803] g
Gyroscope:	 [23.86765, -89.48529, 41.89706] °/s
Accelerometer:	 [0.3968681, 0.3868363, -1.054563] g
Gyroscope:	 [10.04706, 40.17353, 36.69412] °/s
Accelerometer:	 [-0.2840714, 0.4367507, -1.47541] g
Gyroscope:	 [-11.38824, 90.00295, 69.30882] °/s
Accelerometer:	 [-1.105946, 0.6486421, -0.9256178] g
Gyroscope:	 [-3.082353, 43.22647, 25.04706] °/s
Accelerometer:	 [-1.228774, 0.06802055, 0.1641791] g
Gyroscope:	 [96.37353, -49.43824, -9.514706] °/s
Accelerometer:	 [-0.768779, -0.3621238, 0.8649376] g
Gyroscope:	 [96.37353, -86.61765, -9.844118] °/s
Accelerometer:	 [-1.309518, -0.2123807, 2.004343] g
Gyroscope:	 [-95.62353, -95.62353, 60.65883] °/s
Accelerometer:	 [-1.988745, 0.8896502, 1.869342] g
Gyroscope:	 [-95.62353, -95.62353, 6.073529] °/s


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 71, in <module>
KeyboardInterrupt: 
